In [1]:
# ============================================================
# ⚡ Temporal Fusion Transformer — Multi-horizon Forecast
# ✅ 30-min OHLCV + indicators, no overnight gaps
# ============================================================

import os
import json
import numpy as np
import pandas as pd
import torch
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer, QuantileLoss
from pytorch_forecasting.metrics import MAE, RMSE
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import plotly.graph_objects as go

# ---------------------- CONFIG ----------------------
DATA_BASE_PATH = "history_data"
STOCK_CODE = "FORCEMOT"
TIMEFRAME = "30min"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

MODEL_DIR = "models"
os.makedirs(MODEL_DIR, exist_ok=True)
MODEL_FILE = os.path.join(MODEL_DIR, "tft_multistep_best.ckpt")
RESULT_FILE = "tft_results.json"

LOOKBACK = 50           # encoder length
FORECAST_HORIZON = 26   # decoder length
BATCH_SIZE = 64
EPOCHS = 40
LR = 1e-3

print(f"⚙️ Device: {DEVICE}")
print(f"Forecast horizon: {FORECAST_HORIZON} steps ({FORECAST_HORIZON*30/60:.1f} hrs)")

# ---------------------- LOAD DATA ----------------------
def load_csv(stock_code: str, tf: str):
    folder = os.path.join(DATA_BASE_PATH, f"history_data_{stock_code}")
    files = [f for f in os.listdir(folder) if tf in f and f.endswith(".csv")]
    if not files:
        raise FileNotFoundError(f"No CSV found for {stock_code}")
    path = os.path.join(folder, sorted(files)[-1])
    print("📂 Loading:", path)
    df = pd.read_csv(path)
    df["Timestamp"] = pd.to_datetime(df["Timestamp"])
    df = df.sort_values("Timestamp").reset_index(drop=True)
    return df

df = load_csv(STOCK_CODE, TIMEFRAME)

FEATURES = [
    "Open","High","Low","Close","Volume",
    "MA_Fast","MA_Slow","BB_Upper","BB_Lower",
    "MACD","MACD_Signal","MACD_Hist","+DI","-DI","ADX",
    "RSI14","ATR14","atr_pct"
]
df = df.dropna(subset=FEATURES).reset_index(drop=True)

# ---------------------- PREPROCESS ----------------------
# Assign continuous time index ignoring gaps
df["time_idx"] = np.arange(len(df))
df["stock_code"] = STOCK_CODE  # group id
df = df.reset_index(drop=True)

# ---------------------- DATASET SPLIT ----------------------
# we use last part as test
max_encoder_length = LOOKBACK
max_prediction_length = FORECAST_HORIZON

training_cutoff = df["time_idx"].max() - max_prediction_length * 2
print(f"Training cutoff index: {training_cutoff}")

# ---------------------- TimeSeriesDataSet ----------------------
training = TimeSeriesDataSet(
    df[lambda x: x.time_idx <= training_cutoff],
    time_idx="time_idx",
    target="Close",
    group_ids=["stock_code"],
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    time_varying_known_reals=["time_idx"],
    time_varying_unknown_reals=FEATURES,  # unknown future indicators
    target_normalizer=None,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
)

validation = TimeSeriesDataSet.from_dataset(
    training,
    df,
    min_prediction_idx=training_cutoff + 1,
    stop_randomization=True
)

train_loader = training.to_dataloader(train=True, batch_size=BATCH_SIZE, num_workers=2)
val_loader = validation.to_dataloader(train=False, batch_size=BATCH_SIZE, num_workers=2)

# ---------------------- MODEL ----------------------
tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=LR,
    hidden_size=32,
    attention_head_size=4,
    dropout=0.1,
    hidden_continuous_size=16,
    loss=QuantileLoss(),
    log_interval=10,
    reduce_on_plateau_patience=5,
)
print(f"🧠 TFT parameters: {tft.size()/1e3:.1f}k")

# ---------------------- TRAIN ----------------------
early_stop = EarlyStopping(monitor="val_loss", patience=3, mode="min")
lr_logger = LearningRateMonitor()
checkpoint = ModelCheckpoint(dirpath=MODEL_DIR, filename="tft_best", monitor="val_loss", mode="min")

trainer = Trainer(
    max_epochs=EPOCHS,
    accelerator="gpu" if DEVICE == "cuda" else "cpu",
    gradient_clip_val=0.1,
    callbacks=[early_stop, lr_logger, checkpoint],
    enable_progress_bar=True,
    log_every_n_steps=10
)

trainer.fit(tft, train_loader, val_loader)

best_model_path = checkpoint.best_model_path
print(f"🏁 Best model saved at: {best_model_path}")

# ---------------------- PREDICT ----------------------
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)
pred_raw = best_tft.predict(val_loader, mode="raw")
pred_mean = best_tft.predict(val_loader)  # mean prediction

# Extract actual targets (unscaled)
y_true = torch.cat([y[1] for y in iter(val_loader)], dim=0).numpy()
y_pred = pred_mean.numpy()

# ---------------------- METRICS ----------------------
# Compute per-horizon metrics
mae_list, rmse_list, r2_list = [], [], []
H = y_pred.shape[1]
for h in range(H):
    mae_list.append(mean_absolute_error(y_true[:, h], y_pred[:, h]))
    rmse_list.append(np.sqrt(mean_squared_error(y_true[:, h], y_pred[:, h])))
    r2_list.append(r2_score(y_true[:, h], y_pred[:, h]))

# Directional accuracy (first horizon)
prev_closes = df["Close"].iloc[-len(y_pred): -len(y_pred)+len(y_pred)].values
pred_dir = np.sign(y_pred[:, 0] - prev_closes[:len(y_pred)])
true_dir = np.sign(y_true[:, 0] - prev_closes[:len(y_true)])
diracc = np.mean(pred_dir == true_dir)

best_h = int(np.argmin(mae_list))
print(f"✅ Best horizon: {best_h}, MAE={mae_list[best_h]:.3f}, RMSE={rmse_list[best_h]:.3f}, DirAcc={diracc:.3f}")

# ---------------------- SAVE RESULTS ----------------------
results = {
    "Config": {"lookback": LOOKBACK, "horizon": FORECAST_HORIZON, "lr": LR},
    "Metrics": {
        "MAE_all": mae_list,
        "RMSE_all": rmse_list,
        "R2_all": r2_list,
        "DirAcc_h0": diracc,
        "Best_Horizon": best_h,
    }
}
with open(RESULT_FILE, "w") as f:
    json.dump(results, f, indent=2)
print(f"📁 Results saved: {RESULT_FILE}")

# ---------------------- PLOTLY VISUALIZATION ----------------------
timestamps = df["Timestamp"].iloc[-len(y_true):].reset_index(drop=True)
x_idx = np.arange(len(timestamps))

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_idx, y=y_true[:, best_h], mode="lines+markers", name="Actual Close",
    text=timestamps.dt.strftime("%Y-%m-%d %H:%M"),
    hovertemplate="%{text}<br>Actual: %{y:.2f}<extra></extra>"
))
fig.add_trace(go.Scatter(
    x=x_idx, y=y_pred[:, best_h], mode="lines+markers", name="Predicted Close",
    text=timestamps.dt.strftime("%Y-%m-%d %H:%M"),
    hovertemplate="%{text}<br>Pred: %{y:.2f}<extra></extra>"
))
fig.update_layout(
    title=f"{STOCK_CODE} — Temporal Fusion Transformer (h={best_h})",
    xaxis_title="Index (gap-free)", yaxis_title="Close Price",
    hovermode="x unified"
)
fig.show()


d:\ANACONDA\envs\agentic\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚙️ Device: cuda
Forecast horizon: 26 steps (13.0 hrs)
📂 Loading: history_data\history_data_FORCEMOT\FORCEMOT_30min_2024-11-03_to_2025-11-03.csv
Training cutoff index: 947


d:\ANACONDA\envs\agentic\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'loss' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss'])`.
d:\ANACONDA\envs\agentic\Lib\site-packages\lightning\pytorch\utilities\parsing.py:210: Attribute 'logging_metrics' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['logging_metrics'])`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores


🧠 TFT parameters: 107.4k


TypeError: `model` must be a `LightningModule` or `torch._dynamo.OptimizedModule`, got `TemporalFusionTransformer`